In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from opt_einsum import contract
from threadpoolctl import threadpool_limits
from scipy.sparse.linalg import eigsh

from ising import IsingExact, IsingMPO
from mdopt.mps.utils import create_simple_product_state
from mdopt.optimiser.dmrg import DMRG as dmrg

In [ ]:
"""
In this experiment we will use our DMRG optimiser to find the ground state
of an open-bounded transverse field Ising chain. The Hamiltonian reads:
$H = - sum_{i=1}^{N-1} Z_i Z_{i+1} - h * sum_{i=1}^{N} X_i$.
Here, the magnetic field is in the units of the nearest-neighbour ZZ-interaction.
We find the ground state of this Hamiltonian and compute some observables.
"""

In [ ]:
NUM_SITES = 3
H_MAGNETIC = 1.0
ising_exact = IsingExact(num_sites=NUM_SITES, h_magnetic=H_MAGNETIC)
ising_mpo = IsingMPO(num_sites=NUM_SITES, h_magnetic=H_MAGNETIC)
ham_mpo = ising_mpo.hamiltonian_mpo()
m = contract(
    "zabc, adef, dygh -> begcfh",
    ham_mpo[0],
    ham_mpo[1],
    ham_mpo[2],
    optimize=[(0, 1), (0, 1)],
).reshape((8, 8))

print(
    "Checking the exact and the MPO Hamiltonians being the same:",
    (ising_exact.hamiltonian_dense() == m).all(),
)

In [ ]:
print(
    "Checking the ground states from exact diagonalisation and DMRG are the same (up to phase):"
)
print("")

NUM_SITES = 15
H_MAGNETIC = 1.0
NUM_DMRG_RUNS = 10
CHI_MAX = 128
CUT = 1e-9
MODE = "SA"
TOL = 1e-9
ising_exact = IsingExact(num_sites=NUM_SITES, h_magnetic=H_MAGNETIC)
ising_mpo = IsingMPO(num_sites=NUM_SITES, h_magnetic=H_MAGNETIC)
ham_mpo = ising_mpo.hamiltonian_mpo()
ham_sparse = ising_exact.hamiltonian_sparse()

mps_start = create_simple_product_state(NUM_SITES, which="+")

print("DMRG running:")
print("")
engine = dmrg(mps_start, ham_mpo, chi_max=CHI_MAX, cut=CUT, mode=MODE)
engine.run(NUM_DMRG_RUNS)
print("")
ground_state_mps = engine.mps
print("Eigensolver running.")
ground_state_exact = eigsh(ham_sparse, k=2, tol=TOL)[1][:, 0]
print(
    "The ground states are the same:",
    np.isclose(abs(ground_state_mps.dense()), abs(ground_state_exact)).all(),
)

In [ ]:
print("Let us compare the magnetisation plots from exact diagonalisation and DMRG.")
print("The plots should coincide exactly.")
print("")

transverse_magnetic_field_space = np.linspace(0.01, 2.0, 20)
mag_z_exact = []
mag_x_exact = []
mag_z_dmrg = []
mag_x_dmrg = []
for magnetic_field in transverse_magnetic_field_space:
    ising_exact = IsingExact(num_sites=NUM_SITES, h_magnetic=magnetic_field)
    ising_mpo = IsingMPO(num_sites=NUM_SITES, h_magnetic=magnetic_field)
    ham_mpo = ising_mpo.hamiltonian_mpo()
    ham_sparse = ising_exact.hamiltonian_sparse()
    mps_start = create_simple_product_state(num_sites=NUM_SITES, which="+")
    print("DMRG running:")
    print("")
    engine = dmrg(mps_start, ham_mpo, chi_max=CHI_MAX, cut=CUT, mode=MODE)
    engine.run(NUM_DMRG_RUNS)
    print("")
    ground_state_mps = engine.mps
    print("Eigensolver running.")
    print("")
    ground_state_exact = eigsh(ham_sparse, k=2, tol=TOL)[1][:, 0]

    mag_z_exact.append(
        ising_exact.average_chain_z_magnetisation(ground_state_exact)
    )
    mag_x_exact.append(
        ising_exact.average_chain_x_magnetisation(ground_state_exact)
    )

    mag_z_dmrg.append(ising_mpo.average_chain_z_magnetisation(ground_state_mps))
    mag_x_dmrg.append(ising_mpo.average_chain_x_magnetisation(ground_state_mps))

In [ ]:
plt.figure(figsize=(9, 4.5))
plt.plot(transverse_magnetic_field_space, mag_z_exact, label="Exact")
plt.plot(
    transverse_magnetic_field_space, mag_z_dmrg, label="DMRG", linestyle="dashed"
)
plt.xlabel("Transverse magnetic field $h$")
plt.ylabel("Longitudinal magnetisation $m_z$", rotation=90, labelpad=10)
plt.xlim((0, 2))
plt.ylim((-1, 1))
plt.legend(fontsize=10)
plt.tight_layout()
plt.show()

plt.figure(figsize=(9, 4.5))
plt.plot(transverse_magnetic_field_space, mag_x_exact, label="Exact")
plt.plot(
    transverse_magnetic_field_space, mag_x_dmrg, label="DMRG", linestyle="dashed"
)
plt.xlabel("Transverse magnetic field $h$")
plt.ylabel("Transverse magnetisation $m_x$", rotation=90, labelpad=10)
plt.xlim((0, 2))
plt.ylim((0, 1))
plt.legend(fontsize=10)
plt.tight_layout()
plt.show()

In [ ]:
"""
In this experiment we use the DMRG optimiser to find the ground state
of an open-bounded transverse field Ising chain. The Hamiltonian reads:
$H = - sum_{i=1}^{N-1} Z_i Z_{i+1} - h * sum_{i=1}^{N} X_i$.
Here, the magnetic field is in the units of the pairwise Z-interaction.
We find the ground state of this Hamiltonian and compute observables.

Similar to the experiment `ising.py` but computed in parallel.
Co-authored by Moïse Rousseau.
"""

In [ ]:
NUM_SITES = 15


def exact_simulation(magnetic_field: np.float32 = 0):
    """
    Exact simulation function.
    """

    exact = IsingExact(num_sites=NUM_SITES, h_magnetic=magnetic_field)
    ham_sp = exact.hamiltonian_sparse()
    ground_state_ex = eigsh(ham_sp, k=2, tol=1e-9)[1][:, 0]
    return (
        exact.average_chain_x_magnetisation(ground_state_ex),
        exact.average_chain_z_magnetisation(ground_state_ex),
    )


def dmrg_simulation(magnetic_field: np.float32 = 0):
    """
    DMRG simulation function.
    """

    mpo = IsingMPO(num_sites=NUM_SITES, h_magnetic=magnetic_field)
    hamiltonian_mpo = mpo.hamiltonian_mpo()
    start_mps = create_simple_product_state(num_sites=NUM_SITES, which="+")
    dmrg_container = dmrg_optimiser(
        start_mps,
        hamiltonian_mpo,
        chi_max=128,
        cut=1e-9,
        mode="SA",
        silent=False,
        copy=True,
    )
    dmrg_container.run(10)
    dmrg_final_mps = dmrg_container.mps
    return (
        mpo.average_chain_x_magnetisation(dmrg_final_mps),
        mpo.average_chain_z_magnetisation(dmrg_final_mps),
    )


if __name__ == "__main__":

    print(
        "__________________________________________________________________________________________"
    )
    print("")
    print(
        "Checking the ground states from exact diagonalisation and DMRG are the same (up to phase):"
    )
    print("")
    NUM_SITES = 15
    NUM_DMRG_RUNS = 10

    ising_exact = IsingExact(num_sites=NUM_SITES, h_magnetic=1.0)
    ising_mpo = IsingMPO(num_sites=NUM_SITES, h_magnetic=1.0)
    ham_mpo = ising_mpo.hamiltonian_mpo()
    ham_sparse = ising_exact.hamiltonian_sparse()

    mps_start = create_simple_product_state(num_sites=NUM_SITES, which="+")

    print("DMRG running:")
    print("")
    engine = dmrg_optimiser(mps_start, ham_mpo, chi_max=64, cut=1e-9, mode="SA")
    engine.run(NUM_DMRG_RUNS)
    print("")
    ground_state_mps = engine.mps
    print("Eigensolver running.")
    ground_state_exact = eigsh(ham_sparse)[1][:, 0]
    print(
        "The ground states are the same:",
        np.isclose(abs(ground_state_mps.dense()), abs(ground_state_exact)).all(),
    )

    print(
        "__________________________________________________________________________________________"
    )
    print("")
    print("Let us compare the magnetisation plots from exact diagonalisation and DMRG.")
    print("The plots should coincide exactly.")
    print("")

    transverse_magnetic_field_space = np.linspace(0.01, 2.0, 20)
    with threadpool_limits(limits=cpu_count(), user_api="blas"):
        with Pool() as pool:
            print("DMRGs running:")
            print("")
            mag_dmrg = pool.map(
                func=dmrg_simulation, iterable=transverse_magnetic_field_space
            )
            print("")
            print("Eigensolvers running.")
            print("")
            mag_exact = pool.map(
                func=exact_simulation, iterable=transverse_magnetic_field_space
            )
            mag_x_exact = [x[0] for x in mag_exact]
            mag_z_exact = [z[1] for z in mag_exact]
            mag_x_dmrg = [x[0] for x in mag_dmrg]
            mag_z_dmrg = [z[1] for z in mag_dmrg]
            pool.close()
            pool.join()

    plt.figure(figsize=(9, 4.5))
    plt.plot(transverse_magnetic_field_space, mag_z_exact, label="Exact")
    plt.plot(
        transverse_magnetic_field_space, mag_z_dmrg, label="DMRG", linestyle="dashed"
    )
    plt.xlabel("Transverse magnetic field $h$")
    plt.ylabel("Longitudinal magnetisation $m_z$", rotation=90, labelpad=10)
    plt.xlim((0, 2))
    plt.ylim((-1, 1))
    plt.legend(fontsize=10)
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(9, 4.5))
    plt.plot(transverse_magnetic_field_space, mag_x_exact, label="Exact")
    plt.plot(
        transverse_magnetic_field_space, mag_x_dmrg, label="DMRG", linestyle="dashed"
    )
    plt.xlabel("Transverse magnetic field $h$")
    plt.ylabel("Transverse magnetisation $m_x$", rotation=90, labelpad=10)
    plt.xlim((0, 2))
    plt.ylim((0, 1))
    plt.legend(fontsize=10)
    plt.tight_layout()
    plt.show()
